## StockMarketPrediction

#### 1. Data Manipulation

##### 1.1 Loading Data into Database

In [1]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(create_table_sql, conn):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [2]:
conn = create_connection("normalized_ppg6.db")

In [3]:
import os
def loadFiles(foldername='Datasets', fileformat='.csv'):
    csvfiles = list(filter(lambda f: f.endswith(f'{fileformat}'), os.listdir(f'./{foldername}')))
    csvfiles = [os.path.abspath(f"{foldername}/{file}") for file in csvfiles]
    return(csvfiles)

csvfiles = loadFiles()

In [4]:
%%capture
def loadDatabase(file_list=csvfiles, connx=conn, overwrite=True,):
    for filepath in file_list:
        df = pd.read_csv(filepath)
        filename = os.path.basename(filepath)
        table_name = filename.rsplit('.',1)[0]
        write_mode = 'replace' if overwrite else 'append'
        df.to_sql(table_name, connx, if_exists=write_mode, index=False)

loadDatabase()

In [5]:
def query_table(sql_query, connx=conn, ret=False):
    testdf = execute_sql_statement(sql_query, connx)
    if ret:
        return(testdf)
    else:
        display(testdf)

In [6]:
sql_statement = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
query_table(sql_statement,ret=True)

[('AAPL_5Y',),
 ('AMZN_5Y',),
 ('Company',),
 ('FB_5Y',),
 ('GOOG_5Y',),
 ('HistoricalPrices',),
 ('NFLX_5Y',),
 ('TSLA_5Y',)]

In [7]:
sql_statement = """CREATE TABLE HistoricalPrices AS
SELECT * FROM
(SELECT 'AAPL' AS StockSymbol, * FROM AAPL_5Y
UNION ALL
SELECT 'AMZN' AS StockSymbol, * FROM AMZN_5Y
UNION ALL
SELECT 'FB' AS StockSymbol, * FROM FB_5Y
UNION ALL
SELECT 'GOOG' AS StockSymbol, * FROM GOOG_5Y
UNION ALL
SELECT 'NFLX' AS StockSymbol, * FROM NFLX_5Y);"""
create_table(sql_statement, conn)

table HistoricalPrices already exists


#### 2. Data Exploration

#### 3. Time-Series Analysis